In [ ]:
import pandas as pd
import numpy as np

from github import Github
from _private.GithubCredentials import my_credentials

from harpcrawler.harprepo.device import DeviceRepo, TemplateDeviceRepo
from harpcrawler.harprepo.peripheral import PeripheralRepo, TemplatePeripheralRepo
from harpcrawler.gsheets import HarpSpreadsheet

In [ ]:
## Setup
organization = "harp-tech"
gh = Github(login_or_token=my_credentials())

device_template = TemplateDeviceRepo(gh.get_repo(f"{organization}/device.template"))
peripheral_template = TemplatePeripheralRepo(gh.get_repo(f"{organization}/peripheral.template"))

harpSpreadsheet = HarpSpreadsheet("GitHarpCrawler")

In [ ]:
harp_organization = gh.get_organization(organization)

## Get all "Device.*" repositories
device_repos = [DeviceRepo(repo, device_template)\
    for repo in harp_organization.get_repos()\
        if (("device." in repo.full_name) and\
            not("template" in repo.full_name) and\
            not(repo.is_template))]

device_template.run_diagnosis(repos_to_validate=device_repos)
device_diagnosis = device_template.print_diagnosis().applymap(lambda x: str(x))
print(device_diagnosis)

harpSpreadsheet.open_spreadsheet()
harpSpreadsheet.update_spreadsheet("devices", device_diagnosis)


In [ ]:
harp_organization = gh.get_organization(organization)

## Get all "Peripherals.*" repositories

peripheral_repos = [PeripheralRepo(repo, peripheral_template)\
    for repo in harp_organization.get_repos()\
        if (("peripheral." in repo.full_name) and not("template" in repo.full_name))]

peripheral_template.run_diagnosis(repos_to_validate=peripheral_repos)
peripherals_diagnosis = peripheral_template.print_diagnosis().applymap(lambda x: str(x))
print(peripherals_diagnosis)

harpSpreadsheet.open_spreadsheet()
harpSpreadsheet.update_spreadsheet("peripherals", peripherals_diagnosis)